In [3]:
import pandas as pd
import tiktoken
import os
import openai


from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader

In [4]:
df = pd.read_csv("/content/kurslar.csv")

In [5]:
df.head()

,isim,açıklama,tags
0,"Git, GitHub, GitLab ve Jira ile Profesyonel İş...","Git, GitHub, GitLab ve Jira ile İş Akış Yöneti...","['git', 'Github', 'Gİtlab', 'Jira', 'iş akışı'..."
1,Laravel SaaS Uygulaması Geliştirme,Günlük yaşantımızda oldukça fazla online hizme...,"['Laravel', 'SaaS', 'PHP', 'Frontend', 'Backen..."
2,Beautiful Soup ve Selenium ile Web Scrapping,"Bu eğitim, web sitelerinden veri çekme, düzenl...","['HTML', 'Xpath', 'Beautiful Soup', 'Selenium'..."
3,Robotik Kodlama,"Bu eğitim, Arduino tabanlı sistemlerle çalışma...","['Robotik Kodlama', 'Arduino', 'Temel Elektron..."
4,Veri Bilimi ve Makine Öğrenmesi İçin Lineer Cebir,"Günümüz iş dünyasında, Veri Bilimi ve Makine Ö...","['Lineer Cebir', 'Matris', 'Lineer denklem sis..."


In [40]:
df['combined_info'] = df.apply(lambda row: f"isim: {row['isim']}. açıklama: {row['açıklama']} tags: {row['tags']}", axis=1)
df['combined_info'][0:2]

0    isim: Git, GitHub, GitLab ve Jira ile Profesyo...
1    isim: Laravel SaaS Uygulaması Geliştirme. açık...
Name: combined_info, dtype: object

In [8]:
df[['combined_info']].to_csv('df_up.csv', index=False)

In [9]:
loader = CSVLoader(file_path="/content/df_up.csv")
data = loader.load()

In [10]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [ ]:
api_key = "openai_key"

In [11]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
llm = OpenAI(openai_api_key=api_key)

In [14]:
docsearch = Chroma.from_documents(texts, embeddings)

In [25]:
### 1-- benzerlik tabanlı metin eşleştirmesiyle öneri

query = "sql öğrenmek istyiorum?"
docs = docsearch.similarity_search(query, k=1)
if docs:
    result = docs[0].page_content.split("isim:")[1].split(".")[0].strip()
    print(f"--{result}-- sizin için en uygun eğitim olacaktır!")

--MS SQL Sorgulama-- sizin için en uygun eğitim olacaktır!


In [27]:
import os

os.environ['OPENAI_API_KEY'] = api_key

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)


qa = RetrievalQA.from_chain_type(llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=True)

In [28]:
### 2 -- Kulllanıcı bilgisi almadan sadece prompt ile sorgulayarak öneri


from langchain.prompts import PromptTemplate

template = """Sen veribilimi eğitimi veren bir kurumda kişiye özel kurs önerisinde bulunan, kurs tavsiyesi verne bir danışmansın.
Soruyu cevaplamak için söylediğim direktifleri uygula.
Her bir soru için 2 kurs öner.
Eğer sana sorulan soruyu cevaplayamayacaksan gereksiz bir sonuç üretme.

{context}

Question: {question}
Your response:"""



PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}

llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

qa = RetrievalQA.from_chain_type(llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

query = "veri analizi öğrenmek istiyorum. Önce  hangi kursu almalıyım?"
result = qa({'query':query})
print(result['result'])

Veri analizi öğrenmek istiyorsanız, size iki kurs önerim var:

1. Ölçümleme Problemleri: Bu kurs, analistlerin nicel ölçümleri nasıl oluşturup değerlendirebileceklerini öğretir. Farklı ölçüm seviyelerine ve değişkenleri dönüştürmenin yollarına genel bir bakış edinirsiniz. Ayrıca, çalışmaya değer kavramlar için ölçütler geliştirmeyi ve eleştirel olarak değerlendirmeyi öğrenirsiniz. Bu kurs, veri analizi için temel bir altyapı sağlar.

2. Özellik Mühendisliği: Bu kurs, veri temizliği ve değişken mühendisliği gibi veri bilimi süreçlerinin önemli bir kısmını kapsar. Ham veriyi temizleyerek, ilgili veriden yeni özellikler çıkarmayı ve modele girmeye uygun olmayan değişkenleri dönüştürmeyi öğrenirsiniz. Bu kurs, veri setini makine öğrenmesi modellerine girmeye uygun hale getirme konusunda size gerekli becerileri kazandırır.

Bu iki kurs, veri analizi konusunda temel bilgileri edinmeniz için iyi bir başlangıç olacaktır.


In [33]:
### 3 ---Kullanıcı bilgisi ile daha detaylı oalbilecek prompt'lu sorgulayara öneri



from langchain.prompts import PromptTemplate

template_prefix = """Sen veribilimi eğitimi veren bir kurumda kişiye özel kurs önerisinde bulunan, kurs tavsiyesi verne bir danışmansın.
Soruyu cevaplamak için söylediğim direktifleri uygula.
Her bir soru için 2 kurs öner.
Eğer sana sorulan soruyu cevaplayamayacaksan gereksiz bir sonuç üretme.

{context}"""

user_info = """Aşağıdaki bilgiler sana soru soran kullanıcının bilgileri, bu bilgilere en uygun şekilde kurs öner:
Eğitim seviyesi: {Eğitim_Seviyesi}
Programlama bilgisi: {Programlama_seviyesi}"""

template_suffix= """Question: {question}
Your response:"""

user_info = user_info.format(Eğitim_Seviyesi = 'Lisans', Programlama_seviyesi="iyi")

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)



Sen veribilimi eğitimi veren bir kurumda kişiye özel kurs önerisinde bulunan, kurs tavsiyesi verne bir danışmansın. 
Soruyu cevaplamak için söylediğim direktifleri uygula.
Her bir soru için 2 kurs öner. 
Eğer sana sorulan soruyu cevaplayamayacaksan gereksiz bir sonuç üretme.

{context}
Aşağıdaki bilgiler sana soru soran kullanıcının bilgileri, bu bilgilere en uygun şekilde kurs öner:
Eğitim seviyesi: Lisans
Programlama bilgisi: iyi
Question: {question}
Your response:


In [34]:
PROMPT = PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

query = "veri analizi öğrenmek istiyorum. Önce  hangi kursu almalıyım?"
result = qa({'query':query})
print(result['result'])

Veri analizi öğrenmek istiyorsanız, size iki kurs önerim var:

1. Ölçümleme Problemleri: Bu kurs, analistlerin nicel ölçümleri nasıl oluşturup değerlendirebileceklerini öğretir. Bu kurs, veri analizi sürecinde karşılaşabileceğiniz karmaşık kavramları anlamanıza yardımcı olacak ve farklı ölçüm seviyelerine ve değişkenleri dönüştürmenin yollarını keşfetmenizi sağlayacaktır.

2. Özellik Mühendisliği: Bu kurs, veri temizliği ve değişken mühendisliği gibi veri bilimi süreçlerinin önemli bir kısmını kapsar. Bu kurs, ham veriyi temizleyerek, ilgili veriden yeni özellikler çıkarmayı ve modele girmeye uygun olmayan değişkenleri dönüştürmeyi öğretir. Veri setini bir makine öğrenmesi modeline girmeye uygun hale getirme konusunda size gerekli altyapıyı sağlayacaktır.

Bu iki kurs, veri analizi konusunda temel bilgileri edinmeniz için iyi bir başlangıç olacaktır. İyi bir analiz için iyi ölçümler ve veri setinin uygun hale getirilmesi önemlidir. Bu kurslar size bu konularda gerekli becerileri kazand